In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer, QuantoConfig
import torch
from dotenv import load_dotenv
from datasets import load_dataset

data = load_dataset("squad", split="train")
load_dotenv("../.env")

base_model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
quant_config = QuantoConfig(weights="int8")

def fetch_model(is_quant: bool=False):
    config = dict(
        pretrained_model_name_or_path=base_model_path,
        return_dict=True,
        low_cpu_mem_usage=True,
        # torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    if is_quant:
        quant_cfg = {
            "quantization_config": quant_config,
        }
        config = config | quant_cfg

    model = AutoModelForCausalLM.from_pretrained(**config)
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id
    return model

def get_trainable_params(model) -> None:
    """
    Get the number of trainable parameters in the model.
    """
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {params}")

quant_model = fetch_model(is_quant=True)
base_model = fetch_model(is_quant=False)

python-dotenv could not parse statement starting at line 2


In [ ]:
from peft import (
    get_peft_model,
    prepare_model_for_kbit_training,
    LoraConfig
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=[
        "q_proj","k_proj","v_proj",
        "o_proj", "gate_proj", "up_proj",
        "down_proj"
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

lora_base_model = prepare_model_for_kbit_training(base_model)
lora_base_model = get_peft_model(lora_base_model, lora_config)

In [ ]:
get_trainable_params(lora_base_model)

In [ ]:
get_trainable_params(quant_model)

In [ ]:
templated_dataset[0]

In [ ]:
# tokenise the prompts
def tokenize_function(example):
    tokens = tokenizer(
        example['prompt'],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    # Set padding token labels to -100 to ignore them in loss calculation
    tokens['labels'] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokens['input_ids']
    ]
    return tokens

tokenised_dataset = templated_dataset.map(tokenize_function)
tokenised_dataset = tokenised_dataset.remove_columns(["title", "context", "question", "answers", "prompt"])

In [ ]:
# # https://www.mercity.ai/blog-post/guide-to-fine-tuning-llms-with-lora-and-qlora
# # https://huggingface.co/docs/diffusers/en/quantization/quanto
# for name, module in model.named_modules(): # https://www.datacamp.com/tutorial/fine-tuning-llama-3-2
#     print(f"name: {name}")
#     print(f"module: {module}")


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
lora_base_model.train()
training_args = SFTConfig(
    output_dir = "cp",
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 8,
    per_device_eval_batch_size = 8,
    eval_accumulation_steps = 8,
    # optim = "paged_adamw_32bit",
    save_steps = 10,
    logging_steps = 10,
    learning_rate = 5e-5,
    max_grad_norm = 0.3,
    max_steps = 50,
    warmup_ratio = 0.03,
    eval_strategy="steps",
    lr_scheduler_type = "linear",
)
trainer = SFTTrainer(
    model=lora_base_model,
    args=training_args,
    train_dataset=tokenised_dataset["train"],
)

# Initialize Trainer
# trainer = Trainer(
#     model=lora_base_model,
#     args=training_args,
#     train_dataset=tokenised_dataset["train"],
#     eval_dataset=tokenised_dataset["validation"],
#     tokenizer=tokenizer)
#
# # Train the model
# trainer.train()
#
# # Save the model and tokenizer
# trainer.save_model("./fine-tuned-model")
# tokenizer.save_pretrained("./fine-tuned-model")

In [ ]:
import pandas as pd
import json

path = "raw_data/ConvFinQA/train.json"
with open(path, "r") as file:
    data = json.load(file)

In [ ]:
data[0]

In [ ]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
def load_lora_model(base_model_path, adapter_path):
    # First load the base model
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        device_map="auto",
        trust_remote_code=True
    )

    # Then load the LoRA adapter weights
    fine_tuned_model = PeftModel.from_pretrained(base_model, adapter_path)
    return fine_tuned_model, base_model, tokenizer
model_path = "../cp/checkpoint-60"  # Your output directory from training
base_model_path = "meta-llama/Llama-3.2-3B-Instruct"
ft_model, base_model, tokeniser = load_lora_model(base_model_path, model_path)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
val_data = load_dataset("squad")

ft_pipe = pipeline(
    "text-generation",
    model=ft_model,
    tokenizer=tokeniser,
    # torch_dtype=torch.float16,
    device_map="auto",
)

base_pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokeniser,
    # torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
val_data["train"][1]

In [55]:

messages = [{"role": "user", "content": "What is in front of the Notre Dame Main Building?"}]

prompt = tokeniser.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
with torch.no_grad():
    outputs = base_pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

NameError: name 'tokeniser' is not defined

In [ ]:
with torch.no_grad():
    outputs = ft_pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

In [57]:
from train.fine_tune import CustomFineTuner
from transformers import QuantoConfig
from peft import LoraConfig
HUGGING_FACE_MODEL = "meta-llama/Llama-3.2-1B-Instruct"
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

q_cfg = QuantoConfig(weights="int8")
fine_tuner = CustomFineTuner(
    huggingface_model=HUGGING_FACE_MODEL,
    lora_config=lora_cfg,
)

model = fine_tuner.fetch_model()

2025-05-08 22:08:34.026 | INFO     | train.fine_tune:log_trainable_params:98 - Total trainable parameters for meta-llama/Llama-3.2-1B-Instruct: 1235814400


In [58]:
fine_tuner.generate_predictions(model, input="hello")

Device set to use mps


'Hello! How can I assist you today?'

In [62]:
from transformers import pipeline
tokenizer_no_pad = AutoTokenizer.from_pretrained(
    HUGGING_FACE_MODEL,
    add_bos_token=True
)

base_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer_no_pad,
    # torch_dtype=torch.float16,
    device_map="auto",
)

messages = [
    {"role": "user", "content": "Hello"}]

prompt = tokenizer_no_pad.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
with torch.no_grad():
    outputs = base_pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

Device set to use mps
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hello. Is there something I can help you with or would you like to chat?


In [60]:
from mlflow.models import infer_signature
import mlflow
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from transformers import pipeline
os.environ["TOKENIZERS_PARALLELISM"] = "false"
mlflow.set_experiment("Llama-3.2-1B-Instruct")
with mlflow.start_run():
    tokenizer_no_pad = AutoTokenizer.from_pretrained(
        HUGGING_FACE_MODEL,
        add_bos_token=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        HUGGING_FACE_MODEL,
        device_map="auto",

    )
    base_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer_no_pad,
        # torch_dtype=torch.float16,
        device_map="auto",
    )
    # Ensure pad_token_id is an integer before logging
    # if isinstance(model.generation_config.pad_token_id, list):
    #     model.generation_config.pad_token_id = model.generation_config.pad_token_id[0]
    logged_model = mlflow.transformers.log_model(
        transformers_model=base_pipe,
        task="text-generation",
        artifact_path="model",
        signature=infer_signature(
            model_input={
                "content": "What is in front of the Notre Dame Main Building?",
            },
        ),
    )

Device set to use mps
2025/05/08 22:11:22 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run rare-grouse-884 at: http://127.0.0.1:8080/#/experiments/821275711990790082/runs/4f907bbfd18b4475ab15b8a37870b1d6
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/821275711990790082


In [1]:
import mlflow
from dotenv import load_dotenv
load_dotenv("../.env")
loaded_model = mlflow.transformers.load_model(
    "runs:/ca21d1497bb24fb0a74f7542b2b00bac/model",
)

/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025/05/08 23:24:16 INFO mlflow.transformers: 'runs:/ca21d1497bb24fb0a74f7542b2b00bac/model' resolved as 'mlflow-artifacts:/409299786179200491/ca21d1497bb24fb0a74f7542b2b00bac/artifacts/model'
Loading checkpoint shards: 100%|██████████| 9/9 [00:00<00:00, 154.38it/s]
Device set to use mps:0


In [11]:
import torch

messages = [
    {"role": "user", "content": "Hello"}]

prompt = loaded_model.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
with torch.no_grad():
    outputs = loaded_model(prompt, max_new_tokens=120, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [12]:
outputs

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 08 May 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHello<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHello. How can I assist you today?'}]

In [10]:
text = loaded_model.tokenizer.decode(outputs[0]["generated_token_ids"], skip_special_tokens=True)
text

'system\n\nCutting Knowledge Date: December 2023\nToday Date: 08 May 2025\n\nuser\n\nHelloassistant\n\nHello. Is there something I can help you with or would you like to chat?'

In [19]:
loaded_model("hello", do_sample=False)

/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'hello, how can i get a free gift card to a popular restaurant chain?\nHere are some ways to'}]

In [22]:
import torch
messages = [
    {"role": "user", "content": "Write me a sample sql code"}]

prompt = loaded_model.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
with torch.no_grad():
    outputs = loaded_model(prompt, max_new_tokens=120, do_sample=True)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write me a sample sql code<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's a sample SQL query that retrieves data from a table called `employees` with columns `id`, `name`, `department`, and `salary`:

```sql
SELECT 
    id, 
    name, 
    department, 
    salary
FROM 
    employees
ORDER BY 
    id;
```

This query will return a list of all employees, including their names, departments, and salaries, sorted by their `id` in ascending order.

If you want to retrieve only specific columns, you can specify them in the `SELECT` clause:

```sql
SELECT 
   


In [2]:
from datasets import load_dataset

data = load_dataset("squad", split="train")

/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
import pandas as pd

def apply_template(question):
    messages = [{"role": "user", "content": question}]
    return loaded_model.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

data = pd.DataFrame(data)
data["question"] = data["question"].apply(apply_template)
data

,id,title,context,question,answers,processed
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ...",<|begin_of_text|><|start_header_id|>system<|en...
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe...",<|begin_of_text|><|start_header_id|>system<|en...
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'...",<|begin_of_text|><|start_header_id|>system<|en...
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...,<|begin_of_text|><|start_header_id|>system<|en...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...,<|begin_of_text|><|start_header_id|>system<|en...
...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,"{'text': ['Oregon'], 'answer_start': [229]}",<|begin_of_text|><|start_header_id|>system<|en...
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,"{'text': ['Rangoon'], 'answer_start': [414]}",<|begin_of_text|><|start_header_id|>system<|en...
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"{'text': ['Minsk'], 'answer_start': [476]}",<|begin_of_text|><|start_header_id|>system<|en...
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"{'text': ['1975'], 'answer_start': [199]}",<|begin_of_text|><|start_header_id|>system<|en...


['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?',
 'When did the Scholastic Magazine of Notre dame begin publishing?',
 "How often is Notre Dame's the Juggler published?",
 'What is the daily student paper at Notre Dame called?',
 'How many student news papers are found at Notre Dame?',
 'In what year did the student paper Common Sense begin publication at Notre Dame?',
 'Where is the headquarters of the Congregation of the Holy Cross?',
 'What is the primary seminary of the Congregation of the Holy Cross?',
 'What is the oldest structure at Notre Dame?',
 'What individuals live at Fatima House at Notre Dame?',
 'Which prize did Frederick Buechner create?',
 'How many BS level degrees are offered in the College of Engineering at Notre

In [5]:
import requests
from dotenv import load_dotenv
import os
load_dotenv("../.env")
r = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers={
        "Authorization" : "Bearer " + os.getenv("OPENAI_API_KEY"),
        "Content-Type" : "application/json"
    },
    json={
        "model": "gpt-4.1-mini-2025-04-14",
        "messages": [
            {"role": "user", "content": "What is in front of the Notre Dame Main Building?"}
        ],
        "temperature": 0.0,
        "max_tokens": 100,
    }
)
r

<Response [200]>

In [6]:
r.json()

{'id': 'chatcmpl-BV548HBFqWpQxieXd48XhVbfvIVJK',
 'object': 'chat.completion',
 'created': 1746747340,
 'model': 'gpt-4.1-mini-2025-04-14',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'In front of the Notre Dame Main Building (the iconic building on the University of Notre Dame campus in Indiana) is the **Golden Dome**. The Golden Dome is a prominent feature of the Main Building, topped with a statue of the Virgin Mary. Directly in front of the Main Building, you\'ll also find the **Main Circle**, a large circular driveway and green space that serves as a central gathering area on campus. Additionally, the **Word of Life mural** (commonly known as "Touch',
    'refusal': None,
    'annotations': []},
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 18,
  'completion_tokens': 100,
  'total_tokens': 118,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0

In [2]:

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer_no_pad = AutoTokenizer.from_pretrained(model_path, add_bos_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
)
base_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer_no_pad,
    device_map="auto",
    truncation=True,
    max_length=1024,
)

/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps


In [8]:
import torch
message = [
    {"role": "user", "content": "Which NFL team won Super Bowl 50?"},
]
prompt = tokenizer_no_pad.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)
with torch.no_grad():
    outputs = base_pipe(
        prompt,
        do_sample=True,
    )
print(outputs[-1]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Which NFL team won Super Bowl 50?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The Denver Broncos won Super Bowl 50, defeating the Carolina Panthers 24-10.


In [48]:
import mlflow
from dotenv import load_dotenv

load_dotenv("../.env")

mlflow_model = mlflow.transformers.load_model(
    "runs:/57be7000b77448e1891c09bb732a8946/model",
)


2025/05/09 23:52:48 INFO mlflow.transformers: 'runs:/57be7000b77448e1891c09bb732a8946/model' resolved as 'mlflow-artifacts:/409299786179200491/57be7000b77448e1891c09bb732a8946/artifacts/model'
Loading checkpoint shards: 100%|██████████| 9/9 [00:00<00:00, 146.28it/s]
Device set to use mps:0


In [2]:
model_copy = mlflow_model

In [11]:
mlflow_model.model.generation_config.max_new_tokens = 512
mlflow_model.model.generation_config.max_new_tokens = 512
mlflow_model.model.generation_config.temperature = 0
mlflow_model.model.generation_config.do_sample = False

In [5]:
mlflow_model.model.generation_config.max_new_tokens

10

In [67]:
cfg = mlflow_model.model.generation_config.to_dict() | {
    "max_new_tokens": 4,
    "do_sample": True,
}

In [68]:
from transformers import GenerationConfig
mlflow_model.model.generation_config = GenerationConfig(**cfg)

In [12]:
mlflow_model.model.generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "max_new_tokens": 512,
  "temperature": 0,
  "top_p": 0.9
}

In [52]:
pipe = pipeline(
    "text-generation",
    model=mlflow_model.model,
    tokenizer=mlflow_model.tokenizer,
    device_map="auto",
    truncation=True,
    max_new_tokens=512,
)

Device set to use mps:0


In [50]:
import torch
message = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What color was used to emphasize the 50th anniversary of the Super Bowl?"},
]
prompt = mlflow_model.tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)
prompt
# with torch.no_grad():
#     outputs = mlflow_model(
#         prompt
#     )
# print(outputs[-1]["generated_text"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 09 May 2025\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat color was used to emphasize the 50th anniversary of the Super Bowl?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [34]:
import pandas as pd
import json

with open("data/train.json", "r") as file:
    data = json.load(file)
data = pd.DataFrame(data)

In [35]:
exploded = data.explode("qa")
exploded

,pre_text,post_text,filename,table_ori,table,qa,id,annotation,qa_0,qa_1
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",question,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",answer,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",explanation,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",ann_table_rows,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",ann_text_rows,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3036,[table of contents ( 4 ) the increase in cash ...,[( 1 ) represents the rolling three-month aver...,CDW/2017/page_56.pdf,"[[, December 31,], [(in days), 2017, 2016, 201...","[[( in days ), december 31 , 2017, december 31...",steps,Single_CDW/2017/page_56.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
3036,[table of contents ( 4 ) the increase in cash ...,[( 1 ) represents the rolling three-month aver...,CDW/2017/page_56.pdf,"[[, December 31,], [(in days), 2017, 2016, 201...","[[( in days ), december 31 , 2017, december 31...",program,Single_CDW/2017/page_56.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
3036,[table of contents ( 4 ) the increase in cash ...,[( 1 ) represents the rolling three-month aver...,CDW/2017/page_56.pdf,"[[, December 31,], [(in days), 2017, 2016, 201...","[[( in days ), december 31 , 2017, december 31...",gold_inds,Single_CDW/2017/page_56.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
3036,[table of contents ( 4 ) the increase in cash ...,[( 1 ) represents the rolling three-month aver...,CDW/2017/page_56.pdf,"[[, December 31,], [(in days), 2017, 2016, 201...","[[( in days ), december 31 , 2017, december 31...",exe_ans,Single_CDW/2017/page_56.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN


In [14]:
answer_table = pd.json_normalize(data["qa"])
answer_table

,question,answer,explanation,ann_table_rows,ann_text_rows,steps,program,exe_ans,program_re,gold_inds.table_6,...,gold_inds.text_47,gold_inds.table_16,gold_inds.text_49,gold_inds.table_15,gold_inds.text_63,gold_inds.table_14,gold_inds.text_73,gold_inds.text_76,gold_inds.text_211,gold_inds.text_58
0,what was the percentage change in the net cash...,14.1%,,[6],[],"[{'op': 'minus2-1', 'arg1': '206588', 'arg2': ...","subtract(206588, 181001), divide(#0, 181001)",0.14136,"divide(subtract(206588, 181001), 181001)",2008 the net cash from operating activities of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,what was the percent of the growth in the reve...,1.3%,the percent growth of the revenue is the diffe...,[1],[],"[{'op': 'minus2-1', 'arg1': '9362.2', 'arg2': ...","subtract(9362.2, 9244.9), divide(#0, 9244.9)",0.01269,"divide(subtract(9362.2, 9244.9), 9244.9)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,what was the percentage change in net sales fr...,-32%,,[1],[],"[{'op': 'minus1-1', 'arg1': '5363', 'arg2': '7...","subtract(5363, 7983), divide(#0, 7983)",-0.3282,"divide(subtract(5363, 7983), 7983)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,what was the difference in percentage cumulati...,-26.16%,,"[1, 2]",[],"[{'op': 'minus2-1', 'arg1': '75.95', 'arg2': '...","subtract(75.95, const_100), divide(#0, const_1...",-0.2616,"subtract(divide(subtract(75.95, const_100), co...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033,what is the percentage change in gross unpaid ...,1.63%,,"[1, 6]",[],"[{'op': 'minus1-1', 'arg1': '37783', 'arg2': '...","subtract(37783, 37176), divide(#0, 37176)",0.01633,"divide(subtract(37783, 37176), 37176)",( in millions of u.s . dollars ) the balance e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data = pd.concat([data, answer_table], axis=1)

In [22]:
data

,pre_text,post_text,filename,table_ori,table,qa,id,annotation,qa_0,qa_1,...,gold_inds.text_47,gold_inds.table_16,gold_inds.text_49,gold_inds.table_15,gold_inds.text_63,gold_inds.table_14,gold_inds.text_73,gold_inds.text_76,gold_inds.text_211,gold_inds.text_58
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'question': 'what was the percentage change i...,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[substantially all of the goodwill and other i...,[the above unaudited pro forma financial infor...,RSG/2008/page_114.pdf,"[[, Year Ended December 31, 2008 (Unaudited), ...","[[, year ended december 31 2008 ( unaudited ),...",{'question': 'what was the percent of the grow...,Single_RSG/2008/page_114.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[in a new business model such as the retail se...,[.],AAPL/2002/page_23.pdf,"[[, 2002, 2001, 2000], [Net sales, $5,742, $5,...","[[, 2002, 2001, 2000], [net sales, $ 5742, $ 5...",{'question': 'what was the percentage change i...,Single_AAPL/2002/page_23.pdf-1,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[( 1 ) includes shares repurchased through our...,[.],UPS/2009/page_33.pdf,"[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...","[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...",{'question': 'what was the difference in perce...,Single_UPS/2009/page_33.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[( 1 ) includes shares repurchased through our...,[.],UPS/2009/page_33.pdf,"[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...","[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...",NaN,Double_UPS/2009/page_33.pdf,{'amt_table': '<table class='wikitable'><tr><t...,{'question': 'what is the roi of an investment...,{'question': 'what was the difference in perce...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,[contractual obligations we summarize our enfo...,[( 1 ) includes only principal payments owed o...,WRK/2018/page_56.pdf,"[[, Payments Due by Period], [(In millions), T...","[[( in millions ), payments due by period tota...",NaN,Double_WRK/2018/page_56.pdf,{'amt_table': '<table class='wikitable'><tr><t...,{'question': 'what percent of payments are dif...,{'question': 'what would the purchase obligati...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033,[critical accounting estimates our consolidate...,[( 1 ) net of provision for uncollectible rein...,CB/2009/page_81.pdf,"[[, 2009, 2008], [(in millions of U.S. dollars...","[[( in millions of u.s . dollars ), 2009 gross...",{'question': 'what is the percentage change in...,Single_CB/2009/page_81.pdf-1,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3034,[income taxes american water and its subsidiar...,[environmental costs the company 2019s water a...,AWK/2017/page_128.pdf,"[[, 2017, 2016, 2015], [Allowance for other fu...","[[, 2017, 2016, 2015], [allowance for other fu...",NaN,Double_AWK/2017/page_128.pdf,{'amt_table': '<table class='wikitable'><tr><t...,{'question': 'what was the combined amount of ...,{'question': 'what percentage of total afudc i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3035,[transfer agent and registrar for common stock...,"[1 included shares of our common stock , par v...",IPG/2015/page_24.pdf,"[[, Total Number ofShares (or Units)Purchased<...","[[, total number ofshares ( or units ) purchas...",NaN,Double_IPG/2015/page_24.pdf,{'amt_table': '<table class='wikitable'><tr><t...,{'question': 'what is the total cash used for ...,{'q

In [23]:
refined_table = data[["table", "question", "answer", "program_re"]]

In [24]:
refined_table = refined_table.dropna()

In [56]:
df = pd.read_csv("data/val.csv")
df.columns

Index(['table', 'question', 'answer', 'program_re', 'inputs'], dtype='object')

In [42]:
import pandas as pd

# Option 1: Using f-strings in a function
def format_prompt(table, question):
    return f"""
Context: {table}
Question: {question}
"""

# Apply to DataFrame
df["formatted_prompt"] = df.apply(lambda row: format_prompt(row["table"], row["question"]), axis=1)


In [61]:
df.to_dict(orient="records")

[{'table': "[['2017', '2016', '2015', '2014'], ['( in thousands )', '( in thousands )', '( in thousands )', '( in thousands )'], ['$ 12723', '$ 14215', '$ 15794', '$ 442']]",
  'question': 'how much did entergy receive from the money pool between 2014 and 2017 ? ( in thousands $ )',
  'answer': '43174',
  'program_re': 'add(add(add(12723, 14215), 15794), 442)',
  'inputs': "\nContext: [['2017', '2016', '2015', '2014'], ['( in thousands )', '( in thousands )', '( in thousands )', '( in thousands )'], ['$ 12723', '$ 14215', '$ 15794', '$ 442']]\nQuestion: how much did entergy receive from the money pool between 2014 and 2017 ? ( in thousands $ )\n"},
 {'table': "[['new sites ( acquired or constructed )', '2016', '2015', '2014'], ['u.s .', '65', '11595', '900'], ['asia', '43865', '2330', '1560'], ['emea', '665', '4910', '190'], ['latin america', '715', '6535', '5800']]",
  'question': 'what is the total number of new sites acquired and constructed during 2015?',
  'answer': '6535',
  'pro

In [75]:
new_df = pd.DataFrame(df.to_dict(orient="list"))
# new_df
Dataset.from_pandas(new_df)

Dataset({
    features: ['table', 'question', 'answer', 'program_re', 'inputs'],
    num_rows: 211
})

In [65]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="base_model_llama",
)
response = llm.invoke([
    {"role": "system", "content": "je;;p"},
    {"role": "user", "content": "hye"},
])
print(response)




A simple system and human, which seems like a classic setup for a conversation. Here's my response:

Hyre! (Hello!) How are you?


In [76]:
Dataset.from_pandas(new_df)

Dataset({
    features: ['table', 'question', 'answer', 'program_re', 'inputs'],
    num_rows: 211
})

In [88]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="data/train.csv", split="train")

In [89]:
from train.system_prompt import SYSTEM_PROMPT

def apply_message_template(input_dataset: dict) -> dict:
    """Preprocess the input dataset to fit expected format.

    Training data has the following columns:
    ['table', 'question', 'answer', 'program_re', 'inputs']

    See: https://huggingface.co/docs/trl/en/sft_trainer for details on message
    templates.
    """
    message_template = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": input_dataset["inputs"]},
        {"role": "assistant", "content": input_dataset["program_re"]},
    ]
    return {"messages": message_template}

ds = dataset.map(apply_message_template)

In [92]:
type(ds)

datasets.arrow_dataset.Dataset

In [87]:
ds["train"]

Dataset({
    features: ['table', 'question', 'answer', 'program_re', 'inputs', 'messages'],
    num_rows: 1687
})

In [95]:
from mlflow.data.huggingface_dataset import HuggingFaceDataset

hf_ds = HuggingFaceDataset(
    ds=ds,
    source="tes"
)

In [109]:
import os
eval_dataset = load_dataset(
    path="csv",
    data_files="data/val.csv",
)

In [110]:
eval_dataset["train"]

Dataset({
    features: ['table', 'question', 'answer', 'program_re', 'inputs'],
    num_rows: 211
})

In [105]:
eval_dataset

DatasetDict({
    train: Dataset({
        features: ['table', 'question', 'answer', 'program_re', 'inputs'],
        num_rows: 211
    })
})

In [108]:
from datasets import concatenate_datasets

concatenate_datasets(
    [ds["train"], ds["validation"]],
    axis=1,
)

KeyError: "Column train not in the dataset. Current columns in the dataset: ['table', 'question', 'answer', 'program_re', 'inputs', 'messages']"